### Developing optimisation approach using Scipy (rather than Gekko).

In [1]:
from scipy.optimize import minimize, NonlinearConstraint, basinhopping
import time
import numpy as np

Our additional requirements will be:
- optimise expected_value (prob * risk)
- vector/list of parameters (5 skill entries for each worker -> length = 5*N)
- bound contraints [0,1] for each element
- sum of worker contributions (5 elements) <= how many units they have free to contribute 
- departmental workload is met (This is the hard one because it is dynamic: the constraint function will have to be re-evaluated with each assignment tested by the otpimisation routin. *Can Gekko handle this?* - it appears so from the above, but if not we may need to approximate the workload constraint.)
- add team size constraint

In [2]:
import sys, os
MODEL_DIR = os.path.realpath(os.path.dirname('..\superscript_model'))
sys.path.append(os.path.normpath(MODEL_DIR))

In [3]:
from superscript_model import model
from superscript_model.utilities import Random
from superscript_model.project import Project
from superscript_model.organisation import Team
from superscript_model.config import MIN_TEAM_SIZE, MAX_TEAM_SIZE

In [4]:
STEPS = 25

abm = model.SuperScriptModel(worker_count=60, 
                             new_projects_per_timestep=2, 
                             worker_strategy = "Stake",
                             organisation_strategy = 'Basic')

In [5]:
abm.run_model(STEPS)

In [6]:
tracked = abm.datacollector.get_model_vars_dataframe()
tracked.head(STEPS)

,ActiveProjects,RecentSuccessRate,SuccessfulProjects,FailedProjects,NullProjects,WorkersOnProjects,WorkersWithoutProjects,WorkersOnTraining,AverageTeamSize,AverageSuccessProbability,AverageWorkerOvr,AverageTeamOvr,WorkerTurnover,ProjectLoad,TrainingLoad,DeptLoad,Slack,ProjectsPerWorker
0,2,0.000000,0.0,0.0,0,9,51,0,5.500000,0.418061,51.449707,68.221946,0.0,0.055000,0.000000,0.1,0.845000,0.183333
1,3,0.000000,0.0,0.0,1,10,50,0,5.000000,0.441992,51.033801,68.281231,0.0,0.061667,0.000000,0.1,0.838333,0.233333
2,4,0.000000,0.0,0.0,1,13,47,0,5.500000,0.384132,50.668759,68.069255,0.0,0.105000,0.000000,0.1,0.795000,0.350000
3,6,0.000000,0.0,0.0,0,17,43,0,5.166667,0.272324,50.340761,59.581108,0.0,0.160000,0.000000,0.1,0.740000,0.500000
4,8,0.000000,0.0,0.0,0,23,37,0,5.250000,0.233654,50.067947,57.534925,0.0,0.186667,0.000000,0.1,0.713333,0.666667
5,8,0.100000,1.0,1.0,0,22,38,0,5.500000,0.222384,50.289665,56.245237,0.0,0.183333,0.000000,0.1,0.716667,0.700000
6,8,0.069444,0.0,2.0,0,27,33,0,5.750000,0.174645,49.948005,50.455092,0.0,0.176667,0.000000,0.1,0.723333,0.733333
7,7,0.059722,0.0,2.0,1,27,33,0,5.857143,0.185678,49.559520,52.673969,0.0,0.145000,0.000000,0.1,0.755000,0.650000
8,7,0.059722,0.0,2.0,0,24,36,0,5.857143,0.207771,49.279173,53.429054,0.0,0.143333,0.000000,0.1,0.756667,0.616667
9,9,0.059722,0.0,0.0,0,28,30,2,6.111111,0.215694,51.519658,54.019142,25.0,0.231667,0.033333,0.1,0.635000,0.850000


#### Now, with the current state of the system, we will try creating a new project and finding its optimal team allocation:

In [7]:
project = Project(abm.inventory, 
                  abm.inventory.index_total + 1,
                  project_length=5,
                  start_time_offset=abm.inventory.get_start_time_offset(),
                  start_time=abm.schedule.steps)

In [8]:
print(project.requirements.to_string())

{
    "risk": 25,
    "creativity": 3,
    "flexible_budget": false,
    "budget": 103,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 26,
    "hard_skills": {
        "A": {
            "level": 4,
            "units": 3
        },
        "B": {
            "level": 2,
            "units": 4
        },
        "C": {
            "level": 5,
            "units": 7
        },
        "D": {
            "level": 4,
            "units": 5
        },
        "E": {
            "level": 4,
            "units": 7
        }
    }
}


In [9]:
bid_pool = abm.inventory.team_allocator.strategy.invite_bids(project)
print(len(bid_pool))

24


In [10]:
x = np.zeros(len(bid_pool) * 5)
x0 = np.zeros(len(x))
bounds = list([(0,1) for i in x])

In [11]:
print(MAX_TEAM_SIZE) 
print(MIN_TEAM_SIZE)

7
3


#### We now add the constraint on the number uf units that each worker can contribute to the project (worker_unit_budget):

In [13]:
worker_unit_budgets = {
    worker.worker_id: worker.contributions.get_remaining_units(
        project.start_time, project.length
    )
    for worker in bid_pool
}
print(min(worker_unit_budgets.values()))
print(max(worker_unit_budgets.values()))

worker_ids = list(worker_unit_budgets.keys())
worker_unit_budgets = worker_unit_budgets

1
10


In [14]:
worker_unit_budgets

{2: 7,
 3: 6,
 5: 1,
 6: 7,
 7: 2,
 11: 1,
 12: 5,
 13: 7,
 17: 7,
 27: 5,
 36: 6,
 41: 10,
 49: 6,
 62: 5,
 63: 6,
 64: 10,
 80: 1,
 87: 9,
 89: 8,
 96: 9,
 100: 10,
 107: 10,
 110: 9,
 113: 10}

In [15]:
constraints = []
constraints_slsqp = []

for wi, worker in enumerate(bid_pool):
    start = wi*5
    constraints.append(
        NonlinearConstraint(lambda x: sum(np.round(x[start:start+5])) - worker_unit_budgets[worker.worker_id], -np.inf, 0)
    )
    constraints_slsqp.append({
        'type': 'ineq', 
        'fun': lambda x: worker_unit_budgets[worker.worker_id] - sum(np.round(x[start:start+5])),
        'name': 'worker_unit_budget_%d' % worker.worker_id
    })

#### We now add the deparmental workload constraint, which is dynamic and changes with the assignments in the 'x' vector: 

In [16]:
base_dept_unit_budgets = {
            dept.dept_id: dept.get_remaining_unit_budget(
                project.start_time, project.length
            )
            for dept in set(
                [m.department for m in bid_pool]
            )
        }

dept_ids = base_dept_unit_budgets.keys()
base_dept_unit_budgets = base_dept_unit_budgets

In [17]:
dept_members = {
            dept.dept_id: [m for m in bid_pool if m.department.dept_id==dept.dept_id]
            for dept in set(
                [m.department for m in bid_pool]
            )
        }

dept_members = dept_members

In [18]:
base_dept_unit_budgets

{3: 35.0,
 4: 34.0,
 5: 43.0,
 6: 26.0,
 8: 16.0,
 9: 25.0,
 0: 31.0,
 1: 22.0,
 2: 40.0}

In [19]:
def adjusted_dept_unit_budget(x, dept_id):
    
    base = base_dept_unit_budgets[dept_id]
    for worker in dept_members[dept_id]:
        start = bid_pool.index(worker)
        base -= sum(np.round(x[start:start+5]))
        
    return base

In [20]:
for dept_id in dept_ids:
    constraints.append(
        NonlinearConstraint(lambda x: adjusted_dept_unit_budget(x, dept_id), 0, np.inf)
    )
    constraints_slsqp.append({
        'type': 'ineq', 
        'fun': lambda x: adjusted_dept_unit_budget(x, dept_id),
        'name': 'dept_budget_%d' % dept_id
    })

#### Now we add the team size constraint:

In [21]:
def team_size(x):
    
    size = 0
    for wi, worker_id in enumerate(worker_ids):

        start = wi*5
        if sum(np.round(x[start:start+5])) > 0:
            size += 1
            
    return size

In [22]:
constraints.append(NonlinearConstraint(team_size, MIN_TEAM_SIZE, MAX_TEAM_SIZE))

constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: MAX_TEAM_SIZE - team_size(x), 'name': 'team_size_ub'
})
constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: team_size(x) - MIN_TEAM_SIZE, 'name': 'team_size_lb'
})

#### Add bound constraints (COBYLA only):

In [23]:
for i, xi in enumerate(x):
    constraints_slsqp.append({
        'type': 'ineq', 'fun': lambda x: x[i], 'name': 'lb_%d' % i
    })
    constraints_slsqp.append({
        'type': 'ineq', 'fun': lambda x: 1 - x[i] , 'name': 'ub_%d' % i
    })

#### And team budget constraint:

In [24]:
def get_team(x):
    
    SKILLS = ['A', 'B', 'C', 'D', 'E']
    contributions = dict()
    #for skill in project.required_skills:
    for skill in SKILLS:
        contributions[skill] = []
    
    team_members = {}
    for wi, worker_id in enumerate(worker_ids):

        start = wi*5
        in_team = False
        
        #for si, skill in enumerate(project.required_skills):
        for si, skill in enumerate(SKILLS):
            if x[start+si] > 0.5:
                contributions[skill].append(worker_id)
                in_team = True
        
        if in_team:
            team_members[worker_id] = bid_pool[wi]
    
    if len(team_members) == 0:
        return Team(project, {}, None)
    else:
        return Team(project, 
                    team_members, 
                    team_members[list(team_members.keys())[0]],
                    contributions=contributions)    

In [25]:
constraints.append(NonlinearConstraint(int(get_team(x).within_budget()), 1, np.inf))

constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: -1 + int(get_team(x).within_budget()), 
    'name': 'budget_constraint'
})


#### Constrain non-required skill elements to zero:

In [26]:
for si, skill in enumerate(['A', 'B', 'C', 'D', 'E']):
    if skill not in project.required_skills:
        
        for i in range(len(worker_ids)):
            constraints_slsqp.append({
                'type': 'ineq', 'fun': lambda x: 0 - x[i*5 + si], 
                'name': 'non_required_skill_constraint'
            })

In [27]:
constraints = tuple(constraints)

#### Now we define the objective function:

#### Note: we use 0.5 as the boundary to indiciate contributions is non-zero

In [28]:
def objective_func(x):
    
    test_team = get_team(x)
    
    #assert test_team.within_budget()
    
    project.team = test_team
    abm.inventory.success_calculator.calculate_success_probability(project)
    
    return -project.success_probability

In [31]:
# def get_x(team):
    
#     x = np.zeros(len(bid_pool*5))
#     for member_id in team.members:
#         start = 5*[m.worker_id for m in bid_pool].index(member_id)
#         for si, skill in enumerate(['A', 'B', 'C', 'D', 'E']):
#             if skill in project.required_skills:
#                 x[start+si] = 1
                
#     return x

In [32]:
method = 'COBYLA' #'COBYLA' #'SLSQP' # 'trust-constr'
# maxiter = 500
# ftol = 1e-09
# eps=1e-8
# catol = 0.0
# tol=1e-2
# rhobeg = 0.6
# verbose=3
# factorization_method='QRFactorization'
# initial_tr_radius=100

# #guess=list([Random.randint(0,1) for i in range(len(x))])
guess=np.zeros(len(x))
# #guess=np.ones(len(x))
# #guess[1] = 1.0
# #guess[2] = 1.0
# #guess[3] = 1.0
# #guess[4] = 1.0
# #guess[6] = 1.0
# #guess[7] = 1.0
# #guess = get_x(guess_team)

# # For trust-constr: (LinAlgError: SVD did not converge)
# # result_1 = minimize(objective_func, guess, method=method, bounds=bounds, constraints=constraints,
# #                 options={'maxiter': maxiter, 'verbose': verbose, 
# #                          'factorization_method': factorization_method,
# #                          'initial_tr_radius': initial_tr_radius})#,

# # For COBYLA:
# result_1 = minimize(objective_func, guess, method=method,
#                     constraints=constraints_slsqp, options={'maxiter': maxiter, 'disp': True,
#                                                             'catol': catol, 'rhobeg': rhobeg})

In [33]:
# print(team_size(result_1.x))
# print(result_1)
# print(get_team(result_1.x).to_string())

#### Trying basinhopping with COBYLA at each step...

Notes:
- this keeps finding team 1,5,9 with p=0.603
- with niter=1000 it found 5,9,16,31 with p=0.604, took209 seconds
- with niter=10000 it found 1,5,9 with p=0.610, took 2345 seconds (allocation: "B": [5,9],"C": [1,5], "D": [1], "E": [1]
- on second project. with niter=1000, took 629 seconds to find 1,5,6,9. Why same workers?! p=0.647
- on second project. with niter=100, took 67 seconds to find 5,7,63. with p=0.670
- on second project. with niter=100, took 67 seconds to find 1,9,36. with p=0.637
- (attempt 5) third project. with niter=100, took 57 seconds to find 1,3,5,7,17,42,45. with p=0.837
- (attempt 6) third project. with niter=100, took 51 seconds to find 1,3,5,7,17,42,45. with p=0.920 (different contributions)
- (attempt 7) third project. with niter=100, took 47 seconds to find  1,3,5,7,42,59,93. with p=0.72
- (attempt 8) third project. with niter=1000, took 515 seconds to find 1,3,5,7,42,45,65. with p=0.834 
 
- try seeding random number generator in SS.utilties for reproducible results
- the returned value does meet constraints, even if COBYLA can't find a local minimum that meets constraints. 

In [34]:
def test_constraints(x, verbose=False):
    
    for cons in constraints_slsqp:
        assert cons['type'] == 'ineq'
        if cons['fun'](x) < 0:
            if verbose:
                print("Constraint violated: %s" % cons['name'])
            return False
        
    return True

In [35]:
class MyConstraints(object):
    
    def __init__(self):
        self.test = test_constraints
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        return self.test(x)

#### Define bespoke step taking:

(Note: without this the basinhopping algorithm perturbs all vector elements by a random amount and result in the team size ub constraint being violated all the time.)

This step function could be made smarter, e.g. by focusing only on required skills. And by choosing the workers to add/remove based on some hueristic rather than randomly...

In [36]:
class MyTakeStep(object):
    
    def __init__(self, bid_pool=bid_pool, project=project): #, stepsize=0.5):
        #self.stepsize = stepsize
        self.name = 'add/remove workers'
        self.bid_pool = bid_pool
        self.skills = ['A', 'B', 'C', 'D', 'E']
        self.project = project
        
        # team needs at least as many members as the maximum required skill units:
        self.min_team_size = max(
            [self.project.get_skill_requirement(skill)['units']
            for skill in self.project.required_skills]
        )
   
    def __call__(self, x):
        
        # determine how many workers to add and remove to team:
        number_to_add = min(Random.randint(0, MAX_TEAM_SIZE), 
                            len(bid_pool) - team_size(x))
        
        new_size = team_size(x) + number_to_add
        min_remove = max(0, new_size - MAX_TEAM_SIZE)
        max_remove = new_size - MIN_TEAM_SIZE
        if max_remove < min_remove:
            number_to_remove = 0
        else:
            number_to_remove = Random.randint(min_remove, max_remove)
        
        # choose members to add:
        assert len(bid_pool) >= number_to_add + number_to_remove
        current_team = get_team(x)
        
        to_add = []
        new_team_members = list(current_team.members.values())
        inverted = bid_pool[::-1]
        for i in range(number_to_add):
            chosen = Random.choice(inverted) 
            while chosen in new_team_members:
                chosen = Random.choice(inverted) 
            new_team_members.append(chosen)
            to_add.append(chosen)
        
        for a in to_add:
            start = bid_pool.index(a) * 5
            
            required_skill_count = len(project.required_skills)
            add_skills = Random.choices(
                project.required_skills, 
                min(required_skill_count, 
                    worker_unit_budgets[a.worker_id])
            )
            for skill in add_skills:
                si = self.skills.index(skill)
                x[start+si] = 1
        
        # now select and remove required numer of workers:
        to_remove = Random.choices(new_team_members, number_to_remove)
        for r in to_remove:
            start = bid_pool.index(r) * 5
            for i in range(5):
                x[start+i] = 0
        
        return x

#### We will run an ensemble of optimisations for the same project/team allocation problem:

- 100 * niter=100 (~30 minutes total)
- 100 * niter=1000 (~3 minutes each -> 5 hours)
- 10 * niter=10000 (~30 minutes each -> 5 hours)
- 1 * niter=100000 (~5hours each).

For each repeat we will save:
- time taken
- optimal objective function
- team (as string)
- success_calculator.to_string(project)

Also save project and bid_pol.

In [45]:
import pickle
RESULTS_DIR = 'experiments/optimisation/'
project_number = 3
bid_pool_number = 3

niter_repeats = {
    100: 100,
    1000: 100,
    10000: 10,
    100000: 1
}

results = {niter: {'time':[], 'prob': []} 
           for niter in niter_repeats.keys()}

In [46]:
with open(RESULTS_DIR + 'bid_pool_%d.json' % bid_pool_number, 'wb') as ofile:
    pickle.dump(bid_pool, ofile)

In [47]:
with open(RESULTS_DIR + 'project_%d.json' % project_number, 'wb') as ofile:
    pickle.dump(project, ofile)

In [48]:
my_constraints = MyConstraints()
my_takestep = MyTakeStep()

maxiter = 100
catol = 0.0
#tol=1e-2
rhobeg = 0.6
niter = 10000

guess=np.zeros(len(x))

minimizer_kwargs = {"method":'COBYLA',
                    'constraints': constraints_slsqp, 
                    'options': {'maxiter': maxiter, 'disp': True, 
                                'catol': catol, 'rhobeg': rhobeg}}

def print_fun(x, f, accepted):
    print("at minimum %.4f accepted %d" % (f, int(accepted)))
    if accepted:
        print(team_size(x))

In [53]:
for niter in niter_repeats.keys():
    if niter > 100:
        print('niter = %d' % niter)

        for repeat in range(niter_repeats[niter]):
            print('repeat = %d' % repeat)


            start_time = time.time()
            ret = basinhopping(objective_func, guess, 
                               minimizer_kwargs=minimizer_kwargs,
                               niter=niter, seed=70470, 
                               accept_test=my_constraints,
                               take_step=my_takestep)#,
                               #callback=print_fun,)

            elapsed_time = time.time() - start_time  
            print("%d iterations took %.2f seconds" % (niter, elapsed_time))
            results[niter]['time'].append(elapsed_time)
            results[niter]['prob'].append(ret.fun)

            best_team = get_team(ret.x)
            project.team = best_team
            abm.inventory.success_calculator.calculate_success_probability(project)

            with open(RESULTS_DIR 
                      + 'best_team_project_%d_niter_%d_repeat_%d.json' 
                      % (project_number, niter, repeat), 'wb') as ofile:

                pickle.dump(best_team, ofile)

            with open(RESULTS_DIR 
                      + 'prob_summary_project_%d_niter_%d_repeat_%d.json' 
                      % (project_number, niter, repeat), 'wb') as ofile:

                pickle.dump(
                    abm.inventory.success_calculator.to_string(project), 
                    ofile
                )
        print("Mean prob: ", np.mean(results[niter]['prob']))

niter = 1000
repeat = 0
1000 iterations took 365.11 seconds
repeat = 1
1000 iterations took 175.63 seconds
repeat = 2
1000 iterations took 153.79 seconds
repeat = 3
1000 iterations took 142.08 seconds
repeat = 4
1000 iterations took 166.20 seconds
repeat = 5
1000 iterations took 177.70 seconds
repeat = 6
1000 iterations took 179.77 seconds
repeat = 7
1000 iterations took 178.45 seconds
repeat = 8
1000 iterations took 178.89 seconds
repeat = 9
1000 iterations took 177.09 seconds
repeat = 10
1000 iterations took 178.42 seconds
repeat = 11
1000 iterations took 178.79 seconds
repeat = 12
1000 iterations took 178.59 seconds
repeat = 13
1000 iterations took 176.75 seconds
repeat = 14
1000 iterations took 184.10 seconds
repeat = 15
1000 iterations took 178.95 seconds
repeat = 16
1000 iterations took 184.58 seconds
repeat = 17
1000 iterations took 194.56 seconds
repeat = 18
1000 iterations took 170.97 seconds
repeat = 19
1000 iterations took 176.31 seconds
repeat = 20
1000 iterations took 180.

In [55]:
with open(RESULTS_DIR 
                  + 'all_results_project_%d.json' 
                  % project_number, 'wb') as ofile:
                
            pickle.dump(
                results, 
                ofile
            )

#### Testing multiprocessing:

In [4]:
from multiprocessing import Pool
from superscript_model.optimisation import optimise

In [5]:
def opti(x):
    return x*2

In [8]:
if __name__ ==  '__main__': 
    num_processors = 3
    p=Pool(processes = num_processors)
    output = p.starmap(optimise,[(i,1) for i in range(0,3)])
    #output = p.map(opti,[i for i in range(0,3)])
    print(output)

[0, 1, 2]


In [9]:
output

[0, 1, 4]

*attempt_1 (project 1, niter=10000)
* attempt_2  (project 2, niter=1000)
* [attempt_3  (project 2, niter=100)]
* attempt_4  (project 2, niter=100)
* attempt_5  (project 3, niter=100) *New system state. ABM recreated and run for 25 steps. Bid pool of 39.
* attempt_6  (project 3, niter=100). Same team as attempt 5!
* attempt_7 (projet 3, niter=100). Worse.


#### Note that our methods seems to favour lower numbers (1,5,7,9)? Tried new abm to determine if this was to do with specific workers...

# TODO:

- (assign worker contributions in Team! (get_team?))
- test max_team_sze. Increase? Specify equality and check it works - inspect guesses manually

Note: COBYLA ran 'successfully' (before changing 'contributions=contributions' when calling Team()) with maxiter=1000 and ftol=1e-08. But only assigned three elements as non-zero for a project with 6 reuqired skilll units. 

After changing the above, COBYLA also runs 'successfully'...('result')  

'trust-constr' still gives "LinAlgError: SVD did not converge"

'SLSQP' with bespoke bounds (correct?) currently giving exit mode 8 or 5 depending on initial guess.

'COBYLA' finding a solution! But violating constraints so not optimal! Specify bounds as constraints...

#### Note: one option to try is just letting the x vector define team membership (and revert to automatic skill allocation). Then vector is length N.